In [ ]:
import pysam
import numpy as np
import torch
import pandas as pd

from enformer_pytorch import Enformer
from torch.utils.data import Dataset, DataLoader

import h5py
import sys

sys.path.insert(0,'/hpc/compgen/projects/fragclass/analysis/mvivekanandan/script/madhu_scripts')

import config
import sequenceUtils

import importlib
import os

In [ ]:
importlib.reload(config)
importlib.reload(sequenceUtils)

#Set arguments from config file.
arguments = {}
arguments["refGenomePath"] = config.filePaths.get("refGenomePath")
arguments["coordStoreDirectory"] = config.filePaths.get("coordStoreDirectory")
arguments["enformerOutputStoreFile"] = config.filePaths.get("enformerOutputStoreFile")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"The device used is : {device}")

In [ ]:
startIndexList = []
fileNamesList = []

#Store the range of indices for each file in a dictionary. Only open the concerned file when get_item is called for an index.
def createFileNameIndexList():
    coordFilesDirectory = os.fsencode(arguments["coordStoreDirectory"])
    currentIndex = 0
    for file in os.listdir(coordFilesDirectory):
        filename = os.fsencode(file).decode("utf-8")
        startIndexList.append(currentIndex)
        fileNamesList.append(filename)

        numItemsFile = getNumberLinesInFile(filename)
        currentIndex = currentIndex + numItemsFile

    print(f"The current index is {currentIndex}")
    return startIndexList, fileNamesList

def getNumberLinesInFile(filename):
    filePath = arguments["coordStoreDirectory"] + "/" + filename
    with h5py.File(filePath, 'r') as f:
        trainingSamples = f["trainingCoords"][:]
        numSamples = len(trainingSamples)
        return numSamples

"""
Samples from all files are clubbed together to get the final samples set. A given index used by dataloader could belong to any file. startIndexList is a list of the start index of all the 492 files.
Iterate over this startIndexList and find out which position the indexToFind becomes greater than the startIndex.
The file for which index started at this startIndex will be the one containing the indexToFind. So just get the position of the
startIndex which is just below the indexToFind and check in the fileNamesList for this position.
"""
def getFilePositionFromIndex(startIndexList, indexToFind):
   for i, index in enumerate(startIndexList):
      if(indexToFind < index):
         #This means we moved to the next file, so we have to pick the i before that
         filePosition = i-1
         return filePosition

   #this scenario will bever occur unless indexToFind is a negative value.
   return -1

In [ ]:
def getOneHotEncodedSequenceFromCoordinates(coord):
    referenceGenome = pysam.FastaFile(arguments["refGenomePath"])
    coords = (coord[0].decode('UTF-8'), int(coord[1]), int(coord[2]))
    #Get surrounding sequence for feeding into enformer.
    extendedCoordsForEnformer = sequenceUtils.getCoordsForEnformer(coords, referenceGenome)

    #Get the raw sequence using the coordinates and the reference genome.
    cfDnaFragment = sequenceUtils.getSequenceFromCoord(referenceGenome, extendedCoordsForEnformer)

    #One hot encode sequence
    encodedFragment = sequenceUtils.oneHotEncodeSequence(cfDnaFragment)

    encoded_input_sequence = torch.tensor(np.float32(encodedFragment))
    # encoded_input_sequence = encoded_input_sequence.to(device)
    return encoded_input_sequence

#Look into how much h5py content can be compressed. Greater the compression, longer the time needed to read it again.
def storeAsH5pyFile(enformerOutputToStore, labelsToStore):
   print("Inside store as h5py file")
   with h5py.File(arguments["enformerOutputStoreFile"], 'w') as h5_file:
      h5_file.create_dataset("enformerOutput", data=enformerOutputToStore, compression="gzip", compression_opts=8)
      h5_file.create_dataset("labels", data=labelsToStore, compression="gzip", compression_opts=8)
      h5_file.close()

In [ ]:
#Set the model to eval mode first and then send it to cuda. This prevents the GPU node from running out of memory.
enformerModel = Enformer.from_pretrained('EleutherAI/enformer-official-rough', use_checkpointing = True).eval()
enformerModel = enformerModel.to(device)
nbins = 2
ntracks = 5313

def getEnformerPredictions(enformer_model, sequence):
    print("Inside get enformer prediction !!")
    with torch.no_grad():
        pretrained_output = enformer_model(sequence)['human'][:, 448:450, :]

    # print(f"Printing the output shape from enformer {pretrained_output.shape}", flush=True)

    #Combine enformer outputs from 2 bins into a single long output. Each bin, each track is a feature. So the total
    #number of features for training is num_bins * num_tracks_per_bin. All features can be combined in a
    #single 1D tensor array. The other dimension will be the batch size.
    print("Finished enformer prediction")
    batch_size, nbins, ntracks = pretrained_output.shape

    pretrained_output = torch.reshape(pretrained_output, (batch_size, nbins * ntracks))
    return pretrained_output

In [ ]:
class EnformerInputDataset(Dataset):
    def __init__(self):
        print("Initializing the enformer input dataset")
        inputBedFilesDirectory = os.fsencode(arguments["inputBedFileFolder"])
        self.inputDirectory = inputBedFilesDirectory
        self.startIndexList, self.fileNamesList = createFileNameIndexList

    def __getitem__(self, index):
        filePosition = getFileNameFromIndex(self.startIndexList, index)
        filename = self.fileNamesList[filePosition]
        indexWithinFile = index - startIndexList[filePosition]

        filepath = os.path.join(arguments["coordStoreDirectory"].decode("utf-8"), filename)

        with h5py.File(filePath, 'r') as f:
            coord = f['trainingCoords'][indexWithinFile]

            #Each sample should have only one label, it should be a single value instead of a numpy 1D array.The [0] is to make it a single value instead of a numpy array.
            # label = f['trainingLabels'][index][0]
            label = f['trainingLabels'][:][indexWithinFile]

            encoded_input_sequence = getOneHotEncodedSequenceFromCoordinates(coord)

            #For some cases, the coordinates look fine, but the sequence fetched from the fasta file has size 0.
            #If we pass such samples to enformer for predictions, we get Einops error, due to dimension mismatch.
            try:
                assert encoded_input_sequence.shape == (196607,4)

            except:
                print(f"One of the samples did not have the right dimensions. The sample index is {index}, shape is {encoded_input_sequence.shape}")
                return torch.empty((196607,4)), label

            return encoded_input_sequence, label

    def __len__(self):
        with h5py.File(arguments["dataFile"], 'r') as f:
            label = f['trainingLabels'][:]
            return label.shape[0]

In [ ]:
#The function returns 2 numpy arrays. The 1st numpy array is the enformer output for all cfdna fragments. The second numpy array is the array of labels for all cfDNA fragments.
def getEnformerOutput():
    problem_indices = []

    #For each bin from enformer output that is considered, the output array from enformer will have ntracks number of values. So the total number of values in enformer output array for a single cfDNA fragment will be nbins * ntracks.
    enformer_output_size = (1, nbins*ntracks)
    enformer_output_array = np.empty(enformer_output_size)

    enformerInputDataset = EnformerInputDataset()
    enformerInputDataloader = DataLoader(enformerInputDataset, batch_size=16, num_workers=2)

    for i, data in enumerate(enformerInputDataloader, 0):
        print(f"Processing data for batch {i}", flush = True)

        encodedSequence, label = data

        #Encoded sequence will have the dimension of [batch_size, 196607, 4]. Get item would have been called batch_size
        #number of times. Encoded_input_sequence for some batch items might be empty tensors. So iterate over the encodedSequence
        #and remove all the rows which have empty encoded_input_sequence?
        # print(f"Printing the shape of the encoded sequence {encodedSequence.shape}", flush = )
        # print(f"Printing the shape of label {label.shape}")

        encodedSequence = encodedSequence.to(device)

        #The 1st dimension is the batch number. In this case, we have only a single batch, so take the 1st element from the batch dimension directly.
        enformerPrediction = getEnformerPredictions(enformerModel, encodedSequence)

        #the enformer prediction is still in the GPU (since we sent the enformer model and one hot encoded sequence to the GPU. Numpy arrays are not supported in the GPU(GPU probably supports only tensors). So we pass the enformer prediction to CPU and convert it into a numpy array.
        #Detach is used to remove the gradients from the predictions. Gradients are similar to the weights of the model. In our case, we are only interested in the predictions and not the model training, so we remove the gradients to save space.
        enformerPrediction = enformerPrediction.detach().cpu().numpy()
        print(f"Finished processsing batch {i}, enformer prediction shape is {enformerPrediction.shape}")
        storeAsH5pyFile(enformerPrediction, label)

        # enformer_output_array = np.concatenate((enformer_output_array, enformerPrediction))

        # all_inputs = torch.cat(all_inputs, dim = 0)


    # #Because we are row wise concatenating the enformer predictions to an empty array, the 1st row is the empty array
    # #and it consists of only dummy values. Remove the 1st row from the enformer_output_array
    # enformer_output_array = enformer_output_array[1:, :]
    # print(f"Problematic counts are {problem_counts} and size is {len(problem_counts)}")

    return -1, -1

In [ ]:
if __name__ == '__main__':
    enformerOutputArray, labels = getEnformerOutput()
    print(f"Size of enformer output and labels is {enformerOutputArray.shape}, {labels.shape}")
    # storeAsH5pyFile(enformerOutputArray, labels)